In [41]:
# This is the template for the submission. You can develop your algorithm in a regular Python script and copy the code here for submission.

# TEAM NAME ON KAGGLE
# Group33

# GROUP NUMBER
# "group_33"

# TEAM MEMBERS (E-MAIL, LEGI, KAGGLE USERNAME):
# "vstrozzi@ethz.ch", "19-924-596", "vstrozzi" 
# "hdurand@ethz.ch", "18-815-761", "hlneds"
# "nimholz@ethz.ch", "19-929-637", "nadineimholz"

In [42]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from os import listdir
from os.path import isfile, join
import os
import re
import numpy as np
import pandas as pd
import pickle
from scipy.signal import stft
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, multilabel_confusion_matrix
from sklearn.datasets import make_multilabel_classification
import xgboost as xgb 
from scipy.signal import find_peaks, stft, istft
import xgboost as xgb 
from scipy.signal import butter, lfilter
# You may change the mhealth_activity module but your algorithm must support the original version
from mhealth_activity import Recording, Trace, Activity, WatchLocation, Path

pd.options.display.max_seq_items = 5000

# Helpers

## Walking Algorithm

In [60]:
SAMPLING_RATE = 200

def butter_bandpass(lowcut, highcut, fs, order=5):
    return butter(order, [lowcut, highcut], fs=fs, btype='band')

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

def plot_signal(signal, title, ylabel, sampling_rate=SAMPLING_RATE, peaks=[]):
    x = np.linspace(0, len(signal) / sampling_rate, len(signal))
    t = pd.to_datetime(x, unit='s')

    fig, ax = plt.subplots()
    ax.plot(t if len(peaks) == 0 else np.linspace(0, len(signal), len(signal)) , signal)
    ax.set_title(title)
    ax.set_xlabel('Time [min:sec]' if len(peaks) == 0 else "Indexes")
    ax.set_ylabel(ylabel)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%M:%S'))
    
    # Plot peaks if given
    if len(peaks) != 0:
        plt.plot(peaks, signal[peaks], "x", color="green")
    plt.show()

def get_signal_mag(ax, ay, az):

    return (ax**2 + ay**2 + az**2)**0.5

# Get magnitude walk
def get_signal_walk(path):
    recording = Recording(path)

    # Magnitude
    magn = get_signal_mag(recording.data['ax'].values, recording.data['ay'].values,  recording.data['az'].values)
    
    
    return magn, 0

# Clean low frequencies, RFFT since signals contain only real values (+faster)
def algo_walk(acc_phase, label = 0, wind_length= 7.6, mov_avg_wind_s= 0.85, std_thr= 0.15000000000000002, std_wind= 0.65, freq_max_hz= 3.0, prom = 1.65, sampling_rate = SAMPLING_RATE, show=True):
    # Number samples
    N = len(acc_phase)
    
    # Filter too high frequency out
    acc_phase = butter_bandpass_filter(acc_phase, 1, 3, sampling_rate)
    # Eval moving average over window
    mov_avg_wind = int(mov_avg_wind_s*sampling_rate)
    numbers_series = pd.Series(acc_phase)

    windows = numbers_series.rolling(mov_avg_wind, center=True)
    moving_averages = windows.mean()

    # Set first null entries from the list to zero
    moving_averages = moving_averages.fillna(0)
    acc_filt = moving_averages.to_numpy()

    # Plot filered signal
    if show:
        plot_signal(acc_phase[40*sampling_rate:45*sampling_rate], 'Magnitude Acc Original', 'Amplitude')
        plot_signal(acc_filt[40*sampling_rate:45*sampling_rate], 'Magnitude Acc Cleaned', 'Amplitude')

    # Get peaks every windows sec
    pred = 0

    # Walking or movement detection first
    step = int(std_wind*sampling_rate)
    moving = np.empty(N)
    for i in range(step, N - step):
        std_l = lambda val: np.std(val[int(i - step):int(i + step)]) > std_thr
        moving[i + step]  = std_l(acc_phase)
            
    # Copy for first and last
    moving[0:step] = moving[step:2*step]
    moving[-step:]= moving[-2*step:-step]
    if show:
        plot_signal(moving, "Accel Signal", "Moving")
        plot_signal(acc_phase, "Accel Signal original", "Moving")
    
    
    wind_step = int(wind_length*sampling_rate)
    for i in range(0, N, wind_step):
        acc_wind_filt = acc_filt[i:(i+ wind_step)]
        acc_wind_orig = acc_phase[i:(i+ wind_step)]
        moving_wind = moving[i:(i+ wind_step)]

        # Normalize
        # Here you can modify freely the parameters of find peak
        # Minimum distance for our peaks is len_signal_per_minute/max_acc_beat (30 seconds)
        distance = 1/freq_max_hz*sampling_rate
        prominence = (np.max(acc_wind_filt) - np.min(acc_wind_filt))/prom
        peaks, _ = find_peaks(acc_wind_filt, height = np.mean(acc_wind_filt), distance=distance, prominence=prominence)
        
        # Keeps peaks only where energy is not too low
        peaks = [peak for peak in peaks if moving_wind[peak]]
        # Find peaks with double heel, remove half
        pred += len(peaks)
        

        # Plot example in range
        if i == 8*wind_step and show:
            plot_signal(acc_wind_orig, 'Acc Orig from {} to {} s'.format(i*sampling_rate, (i+ 10)*sampling_rate), 'Amplitude', peaks=peaks)
            plot_signal(acc_wind_filt, 'Acc Filt from {} to {} s'.format(i*sampling_rate, (i+ 10)*sampling_rate), 'Amplitude', peaks=peaks)
            plot_signal(moving_wind, 'Acc wrist Filt from {} to {} s'.format(i*sampling_rate, (i+ 10)*sampling_rate), 'Amplitude', peaks=peaks)

    # Print prediction of score if given heart rate ground truth
    if label != 0:   
        print("We have found {} compared to the gt of {} steps".format(pred, label))
    return pred



## Other Algorithms

In [44]:
SENSOR_DIFF_SAMPLE_RATE = ["mx", "my", "mz", "altitude"]
MEASURES = ["mean", "std", "energy", "max_freq"]
SENSOR_CORR = ["ax", "ay", "az", "gx", "gy", "gz", "mx", "my", "mz", "altitude"]
METRICS = ["ax", "ay", "az", "gx", "gy", "gz", "mx", "my", "mz", "altitude", "Steps", "StepsWindow"]
MAX_LENGTH = 919.935


# Get features per sample across windows
def features_per_sample(df):
    mean = df.groupby('Sample').mean()
    mean = mean.add_prefix("mean_")
    std = df.groupby('Sample').std()
    std = std.add_prefix("std_")

    quantile25 = df.groupby('Sample').quantile(q=0.25)
    quantile25 = quantile25.add_prefix("q25_")

    quantile75 = df.groupby('Sample').quantile(q=0.75)
    quantile75 = quantile75.add_prefix("q75_")

    df_final =  pd.concat([mean, std, quantile25, quantile75], axis = 1)
    return df_final 


In [45]:
# Keep only features. Attention, features_corr need to give feats in correct order ex. ax, az, gx, gy, mz
def drop_features(df, features, features_corr, measures):
    corr_feats = [x for x in df.columns for y in features_corr if y in x]
    features_measures = [x for x in df.columns for y in features if y in x]
    measures_drop = [x for x in df.columns for y in measures if y in x]
    return df.drop(features_measures + corr_feats + measures_drop, axis = 1)

In [47]:
# Extend the df to have at least max_window elements per sample, where the padding is with 0
def extend_df_windows(df, data_every = 30, wind_length_s = 60, wind_overlap_per = 0.5):
    wind_step = wind_length_s*wind_overlap_per
    max_window = int((MAX_LENGTH)//wind_step)

    # Create max_window consecutive rows
    df['g'] = df.groupby('Sample').cumcount()
    mux = pd.MultiIndex.from_product([df['Sample'].unique(), range(max_window)], names=('Sample','g'))

    df = (df.set_index(['Sample','g'])
        .reindex(mux, fill_value=0)
        .reset_index(level=1, drop=True)
        .reset_index())
    

    # Reduce number of rows by
    rol = max_window//data_every

    df_label = df[["Sample"]].iloc[rol - 1::rol, :]
    df = df.drop(["Sample"], axis=1).rolling(rol).mean() 

    df = df.iloc[rol - 1::rol, :]
    
    df = pd.concat([df_label, df], axis = 1).reset_index(drop=True)
                
    # Group every max_window rows
    s = df.groupby(['Sample']).cumcount()

    df1 = df.set_index(['Sample', s]).unstack().sort_index(level=1, axis=1)
    df1.columns = [f'{x}{y}' for x, y in df1.columns]
    df1 = df1.reset_index()

    return df1

In [50]:
measures_drop = ["energy"]
features_train_remove = ["Frequency", "path_idx", "watch_loc", "Length", "Steps", "STANDING", "Timestamps", "WALKING",	"RUNNING",	"CYCLING"]

features_watch_remove = ["altitude", "Steps", "StepsWindow", "mx", "my", "mz"]
features_activ_remove = ["altitude", "mx", "my", "mz"]

features_path_remove = ["ax", "ay", "az", "gx", "gy", "gz"]
measures_drop_path = ["energy", "max_freq"]

# Load Models

In [57]:
clf_watch = pickle.load(open("group33_model_watch.pkl", "rb"))
clf_path =  pickle.load(open("group33_model_path.pkl", "rb"))
clf_activ = pickle.load(open("group33_model_activ.pkl", "rb"))

# Inference

In [58]:
# Get the path for all test traces
dir_traces = "./data/test/"#'/kaggle/input/24-exercise2/data/test'

filenames = [join(dir_traces, f) for f in listdir(dir_traces) if isfile(join(dir_traces, f))]
filenames.sort()


In [62]:
# Loop through all filenames to process recordings (uncomment next # to run the code and compute features on the fly (slow))
submission = []
LOW_COR_FEATURES = ['ax_gx_corr', 'ax_gy_corr', 'ax_gz_corr', 'ax_mx_corr', 'ax_my_corr', 'ax_mz_corr', 'ax_altitude_corr', 'ay_gx_corr', 'ay_gy_corr', 'ay_gz_corr', 'ay_mx_corr', 'ay_my_corr', 'ay_mz_corr', 'ay_altitude_corr', 'az_gy_corr', 'az_gz_corr', 'az_mx_corr', 'az_my_corr', 'az_mz_corr', 'az_altitude_corr', 'gx_mx_corr', 'gx_my_corr', 'gx_mz_corr', 'gx_altitude_corr', 'gy_mx_corr', 'gy_my_corr', 'gy_mz_corr', 'gy_altitude_corr', 'gz_mx_corr', 'gz_my_corr', 'gz_mz_corr', 'gz_altitude_corr']
PRELOAD_FEATURES = "group33_features_test.csv"
df_test = pd.read_csv(PRELOAD_FEATURES)

# Remove not highly correlated features
df_test = df_test.drop(LOW_COR_FEATURES, axis=1)

# Compute df for activities
steps = df_test.groupby('Sample').max()[["Steps"]]
df_filt = df_test.drop(["Frequency", "Length", "Timestamps", "Steps"], axis=1)

df_filt = drop_features(df_filt, features_activ_remove, features_activ_remove, measures_drop)
df_features_samples = features_per_sample(df_filt)
X_active =  pd.concat([df_features_samples, steps[["Steps"]]], axis=1)

pred_activ = clf_activ.predict(X_active)

# Compute df for path
steps = df_test.groupby('Sample').max()[["Steps"]]
df_filt = df_test.drop(["Frequency", "Length", "Timestamps", "Steps"], axis=1)

df_filt = drop_features(df_filt, features_path_remove, features_path_remove, measures_drop_path)
# df_features_samples = features_per_sample(df_filt)
df_features_samples = extend_df_windows(df_filt).set_index("Sample")

X_path =  pd.concat([df_features_samples, steps[["Steps"]]], axis=1)

pred_path = clf_path.predict(X_path)

# Compute df for watch
steps = df_test.groupby('Sample').max()[["Steps"]]
df_filt = df_test.drop(["Frequency", "Length", "Timestamps", "Steps"], axis=1)
df_filt = drop_features(df_filt, features_watch_remove, features_watch_remove, measures_drop)

df_features_samples = features_per_sample(df_filt)
X_watch =  pd.concat([df_features_samples, steps[["Steps"]]], axis=1)

pred_watch = clf_watch.predict(X_watch)


#data = pd.DataFrame()
for idx, filename in enumerate(filenames):    
    # Assumes filename format ends with a three-digit ID before ".pkl"
    match = re.search(r'(\d{3})\.pkl$', filename)
    if match:
        id = int(match.group(1))
    else:
        raise ValueError(f'Filename {filename} does not match expected format')    

    # Get number steps
    sign, _ = get_signal_walk(filename)
    nr_steps = algo_walk(sign, show=False)
    exit()

    pred_activ_sample = pred_activ[idx]

    # Get path
    pred_path_sample = pred_path[idx]

    # Get watch loc
    pred_watch_sample = pred_watch[idx]
    
    # Placeholder for the algorithm to process the recording
    # Implement the logic to infer watch location, path index, step count,
    # and activities (standing, walking, running, cycling) here.
    # Ensure your algorithm is tolerant to missing data and does not crash
    # when optional smartphone data traces are missing.

    path_idx = pred_path_sample  # Integer, path in {0, 1, 2, 3, 4}
    watch_loc = pred_watch_sample  # Integer, 0: left wrist, 1: belt, 2: right ankle
    standing = bool(pred_activ_sample[0])  # Boolean, True if participant was standing still throughout the recording
    walking = bool(pred_activ_sample[1])  # Boolean, True if participant was walking throughout the recording
    running = bool(pred_activ_sample[2])  # Boolean, True if participant was running throughout the recording
    cycling = bool(pred_activ_sample[3])  # Boolean, True if participant was cycling throughout the recording
    step_count = nr_steps  # Integer, number of steps, must be provided for each recording

    predictions = {
        'Id': id, 
        'watch_loc': watch_loc, 
        'path_idx': path_idx,
        'standing': standing,
        'walking': walking,
        'running': running,
        'cycling': cycling,
        'step_count': step_count
        }

    submission.append(predictions)

    if idx % 30 == 1:
        print(predictions)
        print("We have elaborated {} samples".format(idx))


{'Id': 1, 'watch_loc': 0, 'path_idx': 4, 'standing': False, 'walking': False, 'running': True, 'cycling': False, 'step_count': 336}
We have elaborated 1 samples


KeyboardInterrupt: 

: 

In [ ]:
# Write the predicted values into a .csv file to then upload the .csv file to Kaggle
# When cross-checking the .csv file on your computer, we recommend using a text editor and NOT excel so that the results are displayed correctly
# IMPORTANT: Do NOT change the name of the columns of the .csv file ("Id", "watch_loc", "path_idx", "standing", "walking", "running", "cycling", "step_count")
submission_df = pd.DataFrame(submission, columns=['Id', 'watch_loc', 'path_idx', 'standing', 'walking', 'running', 'cycling', 'step_count'])
submission_df.to_csv('submission.csv', index=False)